<h1 style="display:inline-block">Slide Bridge</h1><h2>JS Bridge between Google Presentation and a monitor app.</h2>

This is a no-longer-mantained code that used to be hosted on a GCP cloud function that should be called by a JS function added to the Notes in a Google Presentation (using developer tools on chrome). The JS function then send information about the presentation to this CF. Another app was developed to monitor the presentation (it showed a timeline and looking it the presenter could see if he/she were behind schedule).<br/>
<br/>
This CF then was used to create a bridge between Chrome and an external application.

In [ ]:
import google.cloud.storage


def upload_blob( bucket_name, blob_text, destination_blob_name ):
    """Uploads a file to the bucket."""

    storage_client = google.cloud.storage.Client()
    bucket = storage_client.get_bucket( bucket_name )
    blob = bucket.blob( destination_blob_name )

    blob.upload_from_string( blob_text )


def download_blob( bucket_name, blob_name ):
    """Downloads a file from the bucket."""

    storage_client = google.cloud.storage.Client()
    bucket = storage_client.get_bucket( bucket_name )
    blob = bucket.blob( blob_name )

    return blob.download_as_string()


def slide_bridge( request ):
    """Responds to any HTTP request.
    Args:
        request (flask.Request): HTTP request object.
    Returns:
        The response text or any set of values that can be turned into a
        Response object using
        `make_response <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>`.
    """

    # Set CORS headers for the preflight request
    if request.method == 'OPTIONS':
        # Allows GET requests from any origin with the Content-Type
        # header and caches preflight response for an 3600s
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'GET',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }

        return ('', 204, headers)

    # Set CORS headers for the main request
    headers = {
        'Access-Control-Allow-Origin': '*'
    }

    request_json = request.get_json()

    slide = "1"
    read = True

    if request.args and 'slide' in request.args:
        slide = request.args.get( 'slide' )
    if request_json and 'slide' in request_json:
        slide = request_json[ 'slide' ]

    if request.args and 'write' in request.args:
        read = False
    if request_json and 'write' in request_json:
        read = False

    if read == False:
        upload_blob( "active-bolt-265320", slide, "slide-c-function.txt" )

    if read == True:
        return ( download_blob( "active-bolt-265320", "slide-c-function.txt" ), 200, headers )

    return ( "", 200, headers )